In [54]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
from tqdm.notebook import tqdm

from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate

json_data.append({'path': paths[i],
                  'name': name,
                  'type': types_dict[name],
                  'legend': [text],
                  'density_curve_scatter': [x,y],
                  'gauss_approx_plot': [x_gauss, y_gauss],
                  'gauss_approx_data': [{'mus': mus, 'sigmas':sigmas, 'amps':amps}],
                  'angles_series': all_unique_angels,
                  })

In [ ]:
def angles_approx(data, save_name, step, N, M,  save=False,indices=None, font_size=20,scatter_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    alloys_indices=range(len(data))

    if indices is not None:
        alloys_indices=indices

    plt.rcParams['font.size'] = '15'
    plt.figure(figsize=(N, M))

    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1
    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors = ['orange', 'red','blue','green', 'indigo']

    legend=[]
    for i in alloys_indices:
        legend.append(data['data'][i]['name']+' '+data['data'][i]['type'])

    for i in alloys_indices:
        plt.plot(data['data'][i]['gauss_approx_plot'][0], 
                 data['data'][i]['gauss_approx_plot'][1],
                 color=colors[i])

    for i in alloys_indices:
        marker = markers[i]
        plt.scatter(data['data'][i]['density_curve_scatter'][0],data['data'][i]['density_curve_scatter'][1],  marker=marker,color=colors[i],s=scatter_size)

    plt.ylabel('p(x)', fontsize=font_size)
    plt.xlabel('углы, градусы', fontsize=font_size)

    x = [0,60,120,180,240,300,360]
    plt.xticks(x, x)


    if save:
        plt.savefig(f'распределение_углов_{save_name}_шаг_{step}.png')

    for i in alloys_indices:
        print(data['data'][i]['legend'])
    
    plt.legend(legend)
    plt.show()

In [ ]:
step=2
data = open(f'bc_left_(1536,1536,1)_preprocess_step_{step}_angles.json',encoding='utf-8')
data = json.load(data)

In [ ]:
N = 10
M = 7

# save = True
save = False
save_name='test'

angles_approx(data,save_name, step, N, M, indices=[3,0,2],save=save)

In [ ]:
def angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height, font_size=25):
    #
    # аппроксимация каждой моды
    #
    xy_scatter_o = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss_o = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    xy_scatter1 = []
    xy_scatter2 = []

    for j, sc in enumerate(xy_scatter_o):
        x = sc[0]
        y = sc[1]
        xy_scatter1.append([[], []])
        xy_scatter2.append([[], []])
        for i, xi in enumerate(x):
            if xi <= stop1 and xi >= start1:
                xy_scatter1[j][0].append(x[i])
                xy_scatter1[j][1].append(y[i])

            if xi <= stop2 and xi >= start2:
                xy_scatter2[j][0].append(x[i])
                xy_scatter2[j][1].append(y[i])

    plt.rcParams['font.size'] = '25'
    markers = ['v', 's', 'D', 'd', 'p', '*']
    k = 2
    for i in range(len(xy_scatter_o)):
        plt.figure(figsize=(width, height))
        marker = markers[i]

        #    plt.scatter(xy_scatter_o[i][0],xy_scatter_o[i][1])

        x1 = np.array(xy_scatter1[i][0])
        y1 = xy_scatter1[i][1]

        x2 = np.array(xy_scatter2[i][0])
        y2 = xy_scatter2[i][1]

        mu1, sigma1, amp1 = grainApprox.gaussian_fit(x1, y1, mu=120)
        mu2, sigma2, amp2 = grainApprox.gaussian_fit(x2, y2, mu=225)

        x_gauss1 = np.arange(mu1 - 3 * sigma1, mu1 + 3 * sigma1)
        x_gauss2 = np.arange(mu2 - 3 * sigma2, mu2 + 3 * sigma2)

        y_gauss1 = grainStats.gaussian(x_gauss1, mu1, sigma1, amp1)
        y_gauss2 = grainStats.gaussian(x_gauss2, mu2, sigma2, amp2)

        #    plt.plot(x_gauss,y_gauss)
        plt.scatter(x1, y1, marker=marker)
        plt.plot(x_gauss1, y_gauss1)

        plt.scatter(x2, y2, marker=markers[i + 1])
        plt.plot(x_gauss2, y_gauss2)

        #  print('mu1=',mu1,' sigma1=',sigma1,'amp1=',amp1)
        #  print('mu2=',mu2,' sigma2=',sigma2,'amp2=',amp2)
        plt.legend(['moda1', 'moda2', 'approx1', texts[i]], fontsize=14, loc=1)
        plt.ylabel('p(x)', fontsize=font_size)
        plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
        # plt.ylim([0,max(np.log(xy_scatter[i][1][:-1]))])
    # plt.savefig('крупные моды отдельная аппрокс  ' + names[i])

    #  print(texts)

    #  plt.savefig('распределение углов')
    plt.show()

In [ ]:
folder = 'saved np'
width = 12
height = 15

step = 1

start = 340
stop = 360

start1 = 50
stop1 = 180

start2 = 180
stop2 = 300

angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height)